# Импорт библиотек

In [ ]:
!pip install osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 kB 6.6 MB/s eta 0:00:00


In [ ]:
!pip install haversine

In [15]:
import torch  # pytorch
import json  # чтение json файла
import torch.nn as nn
import torch.optim as optim  # регуляризация модели
from torch.nn.utils.rnn import pad_sequence  # выраванивание последовательностей
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from sklearn.model_selection import train_test_split  # разделение выборки на test/train
from sklearn.preprocessing import MinMaxScaler  # нормализация данных

import random

import numpy as np
import osmnx as ox  # библиотека для работы с OSM
import networkx as nx  # для работы с графами местностей
from geopy.distance import geodesic as gd

from haversine import haversine, Unit  # вычисление расстояний между точками кординат
from scipy.interpolate import interp1d  # интерполяция маршрутов

import os
import folium  # для построения html-запросов через leafnet и вывода маршрутов
from sklearn.metrics import mean_squared_error, mean_absolute_error  # метрики для модели

# Рекуррентные нейронные сети

## Класс модели LSTM

Слои LSTM являются двунаправленными, т.е. они могут работать как с данными из прошлого, так и с предсказанными, отсюда для линейного слоя размер скрытого слоя увеличен в 2 раза.

In [3]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        # проходим через LSTM
        out, _ = self.lstm(x)
        # проходим через линейный слой
        out = self.fc(out)
        return out


## Загрузка обученной модели (опционально)

Можно пропустить этап с обучением и сразу загрузить обученную модель (lstm_model.pth) с установившимися параметрами и перейти к шагу с [примером](https://colab.research.google.com/drive/1WcNZ3E7X6JpAAkMWuinQIQ_PZWZPP2h1#scrollTo=El_K5tZg1AuN&line=1&uniqifier=1).

In [ ]:
!gdown 16Hbvud9yHTT22XZ4WLZ0g4JfWkjsikx1

Downloading...
From: https://drive.google.com/uc?id=16Hbvud9yHTT22XZ4WLZ0g4JfWkjsikx1
To: /content/lstm_model.pth
100% 546k/546k [00:00<00:00, 13.9MB/s]


## Параметры модели

Для модели были выбраны следующие параметры:
- входной и выходной размеры = 2, т.к. последовательность имеет размерность N x 2, т.е. ширина и долгота
- размер скрытого слоя LSTM = 64
- число слоёв LSTM = 2
- размер одного батча (отрезка) при обучении модели = 128

Другие характеристики:
- В качестве предобработки данные были нормализованы в интервале от -1 до 1 для более эффективной работы функции активации в виде гиперболического тангенса, содержащейся в слое LSTM.
- В качестве функции ошибки была выбрана среднеквадратическая ошибка (MSE).
- Для регуляризации был выбран оптимизатор AdamW.

In [4]:
sc = MinMaxScaler(feature_range=(-1, 1))  # для нормализации данных от -1 до 1

# Параметры модели
input_size = 2
hidden_size = 64
output_size = 2
num_layers = 2
batch_size = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# создаём модель
model = LSTMModel(input_size, hidden_size, output_size, num_layers)

# функция потерь и оптимизатор
optimizer = optim.AdamW(model.parameters(), lr=0.01)

loss_fn = nn.CTCLoss()

## Загрузка и предобработка данных

Подгрузка выборки в виде последовательностей координат в количестве 10000 штук (routes.json)

In [6]:
!gdown 1VYI0Mi5XTASGDon33RNMjdMlkCKynA0k

"gdown" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


Загрузка данных выборки из файла

In [5]:
# Загрузка маршрутов
routes_path = "training data\\routes.json"
with (open(routes_path, 'r', encoding='utf-8')) as f:
    data = json.load(f)

X, y = data['X'], data['y']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8,
                                                    random_state=42)

Происходит выравнивание последовательностей в соответствии с самой длинной последовательностью и их преобразование в тензоры, далее следует нормализация в отрезок [-1; 1]. После выравнивания лишние элементы будут иметь координаты (0, 0). Затем данные загружаются в dataset и dataloader

(*стоит дополнительно заняться эмбеддингом лишних элементов, чтобы модель их могла не учитывать*)

In [6]:
X_train = [torch.tensor(sc.fit_transform(seq), dtype=torch.float32) for seq in X_train]
y_train = [torch.tensor(sc.fit_transform(seq), dtype=torch.float32) for seq in y_train]
X_train_pad = pad_sequence(X_train, batch_first=True)
y_train_pad = pad_sequence(y_train, batch_first=True)

X_test = [torch.tensor(sc.fit_transform(seq), dtype=torch.float32) for seq in X_test]
y_test = [torch.tensor(sc.fit_transform(seq), dtype=torch.float32) for seq in y_test]
X_test_pad = pad_sequence(X_test, batch_first=True)
y_test_pad = pad_sequence(y_test, batch_first=True)

train_dataset = TensorDataset(X_train_pad, y_train_pad)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test_pad, y_test_pad)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Обучение модели

На обучение выделено 10 эпох.

In [ ]:
num_epochs = 10  # Количество эпох при обучении
train_hist = []
test_hist = []
for epoch in range(num_epochs):
    total_loss = 0.0
    model.train()
    for batch_X, batch_y in train_loader:  # выборка разделяется на части (батчи)
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        predictions = model(batch_X)
        loss = loss_fn(predictions, batch_y)  # для каждого батча считается функция потерь

        # обратное распространение ошибки
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    train_hist.append(average_loss)

    # расчёты для тестовых бачтей
    model.eval()
    with torch.no_grad():
        total_test_loss = 0.0

        for batch_X_test, batch_y_test in test_loader:
            batch_X_test, batch_y_test = batch_X_test.to(device), batch_y_test.to(device)
            predictions_test = model(batch_X_test)
            test_loss = loss_fn(predictions_test, batch_y_test)

            total_test_loss += test_loss.item()

        average_test_loss = total_test_loss / len(test_loader)
        test_hist.append(average_test_loss)

    print(
        f'Epoch [{epoch + 1}/{num_epochs}] - Training Loss: {average_loss:.4f}, Test Loss: {average_test_loss:.4f}')


Epoch [1/10] - Training Loss: 0.0203, Test Loss: 0.0040
Epoch [2/10] - Training Loss: 0.0040, Test Loss: 0.0037
Epoch [3/10] - Training Loss: 0.0037, Test Loss: 0.0036
Epoch [4/10] - Training Loss: 0.0036, Test Loss: 0.0035
Epoch [5/10] - Training Loss: 0.0035, Test Loss: 0.0034
Epoch [6/10] - Training Loss: 0.0035, Test Loss: 0.0034
Epoch [7/10] - Training Loss: 0.0033, Test Loss: 0.0033
Epoch [8/10] - Training Loss: 0.0032, Test Loss: 0.0032
Epoch [9/10] - Training Loss: 0.0030, Test Loss: 0.0030
Epoch [10/10] - Training Loss: 0.0029, Test Loss: 0.0028


Параметры модели можно сохранить в отдельном файле

In [ ]:
torch.save(model, './lstm_model.pth')

## Пример использования

Загрузка файла конфигурации (config.json).

In [ ]:
!gdown 1uTIYTjQHau5R0kOhQG3lspQ-EA16SHxp

Downloading...
From: https://drive.google.com/uc?id=1uTIYTjQHau5R0kOhQG3lspQ-EA16SHxp
To: /content/config.json
100% 163/163 [00:00<00:00, 397kB/s]


### Класс генератора маршрутов

In [ ]:
class RouteGenerator:
    def __init__(self, config_path: str = "config.json", **kwargs):
        """Конструктор класса, которому в именованных аргументах передаётся
        либо название местности, либо точные координаты местности.
        Если было передано название, то происходит обращение к базе данных OSM,
        где потом извлекаются точные координаты. На основе координат строится граф дорог.


        Args:
            config_path (str, optional): Путь к файлу конфигурации. По умолчанию стоит "config.json".

        Raises:
            Exception: Не были переданы ни название местности, ни его координаты.
        """

        self.__load_config(config_path)
        self.data = {'X': [], 'y': []}

        if "place_name" in kwargs.keys():
            self.__place_bbox = list(
                ox.geocode_to_gdf(kwargs["place_name"]).geometry.total_bounds
            )

        elif "place_bbox" in kwargs.keys():
            self.__place_bbox = kwargs["place_bbox"]
        else:
            raise Exception(
                "Укажите название места согласно базе данных OSM либо координаты местности."
            )

        self.graph = ox.graph_from_bbox(self.__place_bbox, network_type="drive")  # Граф дорог местности

    def __load_config(self, file_path: str) -> None:
        """Загрузка данных о константах через файл конфигурации.

        Args:
            file_path (str): Путь к файлу конфигурации.
        """

        with open(file_path, "r") as file:
            config = json.load(file)
            self.__data_amount = config["data_amount"]  # Размер генерируемой выборки
            self.__min_segment = config[
                "min_segment"
            ]  # Минимальное значение отрезка для создания отклонения
            self.__max_segment = config[
                "max_segment"
            ]  # Максимальное значение отрезка для создания отклонения
            self.__min_offset = config["min_offset"]  # Минимальное отклонение
            self.__max_offset = config["max_offset"]  # Максимальное отклонение
            self.__max_route_len = config["max_route_len"]
            self.__min_route_len = config["min_route_len"]

    def save_false_route(self, main_route: list) -> tuple:
        """Генерация одного искажённого маршрута на основе исходного.

        Args:
            main_route: (list): Исходный маршрут.

        Returns:
            Tuple[nx.Graph, list]: Кортеж, внутри которого помещён изменённый граф и полученный маршрут.
        """

        path = main_route
        G = self.graph.copy()
        new_nodes = [path[0]]

        for i in range(len(path) - 1):
            # Начальная и конечная точки отрезка
            u, v = path[i], path[i + 1]
            point1 = (G.nodes[u]["y"], G.nodes[u]["x"])
            point2 = (G.nodes[v]["y"], G.nodes[v]["x"])

            # Расстояние между узлами
            edge_length = gd(point1, point2).meters
            direction_bearing = ox.bearing.calculate_bearing(
                point1[0], point1[1], point2[0], point2[1]
            )

            # Добавление точек через случайное расстояние между 20 и 60 метров
            current_dist = 0
            previous_node = u
            while current_dist < edge_length:
                # Случайное расстояние до следующей точки
                random_dist = random.uniform(self.__min_segment, self.__max_segment)
                current_dist += random_dist

                if current_dist >= edge_length:
                    break

                # Вычисление промежуточной точки
                new_point = gd(meters=current_dist).destination(
                    point1, direction_bearing
                )
                new_lat, new_lon = new_point.latitude, new_point.longitude

                # Случайное отклонение влево или вправо
                offset_direction = direction_bearing + (
                    90 if random.choice([True, False]) else -90
                )
                offset_dist = random.uniform(self.__min_offset, self.__max_offset)
                offset_point = gd(meters=offset_dist).destination(
                    (new_lat, new_lon), offset_direction
                )
                offset_lat, offset_lon = (offset_point.latitude, offset_point.longitude)

                # Добавление новой вершины и её координат
                new_node = max(G.nodes) + 1
                G.add_node(new_node, y=offset_lat, x=offset_lon)
                new_nodes.append(new_node)

                # Добавление ребра между новой точкой и предыдущей точкой
                G.add_edge(previous_node, new_node, length=random_dist)
                G.add_edge(
                    new_node, v, length=edge_length - current_dist
                )  # Связь с основным маршрутом

                previous_node = new_node  # Сместить начальную точку для следующего шага
            new_nodes.append(path[i + 1])

        false_route = [(G.nodes[n]["x"], G.nodes[n]["y"]) for n in new_nodes]
        return G, false_route

    def save_main_route(self) -> tuple:
        """Генерация и сохранение исходного маршрута

        Args: _
        """
        keys = list(self.graph.nodes.keys()).copy()
        node_ids = []

        while len(node_ids) < self.__min_route_len or len(node_ids) > self.__max_route_len:
            try:
                start = random.choice(keys)
                keys.remove(start)
                end = random.choice(keys)
                # Поиск кратчайшего пути
                node_ids = nx.astar_path(self.graph, start, end, weight="length")
            except nx.NetworkXNoPath:
                pass
        main_route = [(self.graph.nodes[n]["x"], self.graph.nodes[n]["y"])
                      for n in node_ids]
        return node_ids, main_route

    @staticmethod
    def calculate_cumulative_distances(route: "np.ndarray"):
        distances = [0]  # Начинаем с 0 элемента
        for i in range(1, len(route)):
            lon1, lat1 = route[i - 1]
            lon2, lat2 = route[i]
            distance = haversine((lon1, lat1), (lon2, lat2), unit=Unit.METERS)
            distances.append(distances[-1] + distance)
        return np.array(distances)

    # Функция для интерполяции маршрута

    def make_equal(self, route: list, num_points: int) -> list:
        route = np.array(route)
        # Вычисляем кумулятивное расстояние
        distances = self.calculate_cumulative_distances(route)

        # Создаем интерполяционные функции для широты и долготы
        interpolation_func_lon = interp1d(distances, route[:, 0], kind='linear')
        interpolation_func_lat = interp1d(distances, route[:, 1], kind='linear')

        new_distances = np.linspace(0, distances[-1], num_points)

        new_lon = interpolation_func_lon(new_distances)
        new_lat = interpolation_func_lat(new_distances)
        new_route = list(np.column_stack((new_lon, new_lat)))
        new_route = [tuple(point) for point in new_route]
        return new_route

    def save_data(self) -> None:
        for i in range(self.__data_amount):
            route_ids, main_route = self.save_main_route()
            _, false_route = self.save_false_route(route_ids)

            main_route = self.make_equal(main_route, len(false_route))
            self.data['y'].append(main_route)
            self.data['X'].append(false_route)
            if (i + 1) % 100 == 0:
                print(f"Сделано {i + 1}/{self.__data_amount} маршрутов")


### Функции для запуска примера

В качестве примера будут созданы 3 html-файла в папке example:
- *input* - входной маршрут
- *target* - целевой маршрут
- *predict* - предсказанный моделью маршрут

In [ ]:
def save_route(points: list, save_folder: str, name: str) -> None:
    # Создаем карту, центрированную на первой точке
    points = [(point[1], point[0]) for point in points]
    plot = folium.Map(location=points[0], zoom_start=15)

    # Соединяем точки линией (маршрут)
    folium.PolyLine(points, color="red", weight=2, opacity=1).add_to(plot)

    # Сохраняем карту в HTML-файл и открываем его
    plot.save(f"{save_folder}/{name}.html")


def lstm_test(save_folder: str) -> None:
    os.makedirs(save_folder, exist_ok=True)

    test_model = torch.load('lstm_model.pth', weights_only=False)
    test_model.eval()

    sc = MinMaxScaler(feature_range=(-1, 1))

    place_bbox = [39.0296, 51.7806, 39.3414, 51.5301]
    generator = RouteGenerator(place_bbox=place_bbox)
    G, result = generator.graph, generator.save_main_route()
    main_ids, main_coords = result
    G_false, false_coords = generator.save_false_route(main_ids)

    main_coords = generator.make_equal(main_coords, len(false_coords))

    save_route(main_coords, save_folder, "target")
    save_route(false_coords, save_folder, "input")

    false_coords = torch.tensor(sc.fit_transform(false_coords), dtype=torch.float32)
    with torch.no_grad():
        predict = test_model(false_coords)
    predict = sc.inverse_transform(predict.detach().numpy())
    print(predict)
    save_route(predict, save_folder, "predict")
    print(f"MSE: {mean_squared_error(predict, main_coords)} \t MAE: {mean_absolute_error(predict, main_coords)}")



Запустим пример и сохраним файлы в папку example.

In [ ]:
lstm_test("example")

[[39.244995 51.673054]
 [39.24536  51.675926]
 [39.24519  51.67606 ]
 [39.24539  51.675865]
 [39.245564 51.675636]
 [39.245365 51.675484]
 [39.24521  51.67535 ]
 [39.24528  51.675312]
 [39.245274 51.675266]
 [39.245483 51.675396]
 [39.245926 51.6756  ]
 [39.246265 51.67552 ]
 [39.24671  51.67573 ]
 [39.2469   51.675735]
 [39.246994 51.675697]
 [39.2473   51.675816]
 [39.247784 51.675865]
 [39.248158 51.675728]
 [39.248726 51.675842]
 [39.249126 51.675797]
 [39.249634 51.675816]
 [39.24995  51.67579 ]
 [39.25026  51.67569 ]
 [39.25054  51.675518]
 [39.250885 51.6752  ]
 [39.25148  51.67513 ]
 [39.25173  51.67488 ]
 [39.251755 51.674442]
 [39.25151  51.67383 ]
 [39.251686 51.67325 ]
 [39.251583 51.672787]
 [39.25119  51.67222 ]
 [39.25105  51.671696]
 [39.25113  51.67133 ]
 [39.251095 51.670864]
 [39.25066  51.67031 ]
 [39.250587 51.66987 ]
 [39.250614 51.669373]
 [39.25038  51.668877]
 [39.250023 51.66836 ]
 [39.25007  51.667953]
 [39.2498   51.667606]
 [39.249443 51.66718 ]
 [39.249542

# Графовые нейронные сети

# Ансамбли моделей

In [17]:
from torch_geometric.nn import GATConv
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
import geopandas as gpd

In [8]:
class GATxRNN(nn.Module):
    """"
    Класс модели, которая использует графовую нейронную сеть с механизмом внимания
    и рекуррентную нейронную сеть на основе seq2seq
    """

    def __init__(self, hidden_dim):
        super().__init__()
        # Кодировщик координат (преобразует (lat, lon) в эмбеддинг)
        self.coord_encoder = nn.Linear(2, hidden_dim)

        # Кодировщик графа (GNN)
        self.gnn = GATConv(hidden_dim, hidden_dim)

        # Попробовать несколько слоёв GAT
        """
        layers = [GATConv(input_dim, hidden_dim, edge_dim=edge_dim)]
        for _ in range(num_layers - 2):
            layers.append(GATConv(hidden_dim, hidden_dim, edge_dim=edge_dim))
        layers.append(GATConv(hidden_dim, output_dim, edge_dim=edge_dim))

        self.convs = torch.nn.ModuleList(layers)
        self.dropout = dropout
        """

        # Механизм внимания между точками и графом
        self.attention = nn.MultiheadAttention(hidden_dim, num_heads=4)

        # Seq2seq, предсказывает следующий узел
        self.decoder = nn.LSTM(hidden_dim, hidden_dim)

    def forward(self, graph_data, route_coords):
        route_emb = self.coord_encoder(route_coords)  # [seq_len, hidden_dim]
        node_emb = self.gnn(graph_data.x, graph_data.edge_index)  # [num_nodes, hidden_dim]

        # 3. Сопоставляем точки маршрута с узлами графа через внимание
        corrected_emb, _ = self.attention(
            route_emb, node_emb, node_emb
        )

        # 4. Декодируем последовательность узлов
        output, _ = self.decoder(corrected_emb)
        return output  # [seq_len, hidden_dim]


In [9]:
# Параметры модели
hidden_dim = 32
batch_size = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# создаём модель
model = GATxRNN(hidden_dim).to(device)

# оптимизатор
optimizer = optim.Adam(model.parameters(), lr=0.01)

# в качестве функции потерь - СТС, так как последовательности разнородной длины
loss_fn = nn.CTCLoss()

In [10]:
# Загрузка графа дорог Воронежа
datapath = "training data"

# Атрибуты вершины: координаты, id, кол-вол улиц
nodes = gpd.read_file(os.path.join(datapath, "nodes.csv"), encoding="utf8")
nodes = nodes.iloc[:, :4]
nodes_type = {'osmid': 'int64', 'street_count': 'int64',
              'y': 'float64', 'x': 'float64'}
nodes = nodes.astype(nodes_type)

edges = gpd.read_file(os.path.join(datapath, "edges.csv"), encoding="utf8")
edge_index = edges[['u', 'v']]
edge_index_type = {'u': 'int64', 'v': 'int64'}
edge_index = edge_index.astype(edge_index_type)

# Атрибуты ребра: id, кол-во полос, односторонность, реверсивность, длина
edge_attr = edges[['oneway', 'reversed', 'length']]
edge_attr_type = {'oneway': bool, 'reversed': bool, 'length': 'float32'}
edge_attr = edge_attr.astype(edge_attr_type)
edge_attr['oneway'] = edge_attr['oneway'].astype(int)
edge_attr['reversed'] = edge_attr['oneway'].astype(int)

In [20]:
nodes_t = torch.tensor(nodes.values)
edge_index_t = torch.tensor(edge_index.values)
edge_attr_t = torch.tensor(edge_attr.values)
graph = Data(x=nodes_t, edge_index=edge_index_t, edge_attr=edge_attr_t)

In [21]:
graph = [graph]
train_loader = ConcatDataset([graph, train_dataset])
test_loader = ConcatDataset([graph, test_dataset])

In [26]:
for some in train_loader:
    print(some)

Data(x=[7352, 4], edge_index=[19200, 2], edge_attr=[19200, 3])
(tensor([[0.8563, 0.5655],
        [0.8631, 0.5521],
        [0.8794, 0.5361],
        ...,
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000]]), tensor([[0.8591, 0.5720],
        [0.8684, 0.5649],
        [0.8776, 0.5578],
        ...,
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000]]))
(tensor([[-1.0000,  0.9674],
        [-0.9831,  0.9736],
        [-0.9807,  0.9857],
        ...,
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000]]), tensor([[-1.0000,  0.9708],
        [-0.9901,  0.9854],
        [-0.9802,  1.0000],
        ...,
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000]]))
(tensor([[-1.0000, -0.9083],
        [-0.9984, -0.8983],
        [-0.9802, -0.8993],
        ...,
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000]]), tensor([[-1.0000, -0.9166],
        [-0.992

KeyboardInterrupt: 

Остаётся только загрузить граф в dataloader/dataset, чтобы можно было его передавать модели

In [25]:
num_epochs = 50  # Количество эпох при обучении
train_hist = []
test_hist = []
for epoch in range(num_epochs):
    total_loss = 0.0
    model.train()
    for batch_X, batch_y in train_loader:  # выборка разделяется на части (батчи)
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        predictions = model(batch_X)
        loss = loss_fn(predictions, batch_y)  # для каждого батча считается функция потерь

        # обратное распространение ошибки
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    train_hist.append(average_loss)

    # расчёты для тестовых бачтей
    model.eval()
    with torch.no_grad():
        total_test_loss = 0.0

        for batch_X_test, batch_y_test in test_loader:
            batch_X_test, batch_y_test = batch_X_test.to(device), batch_y_test.to(device)
            predictions_test = model(batch_X_test)
            test_loss = loss_fn(predictions_test, batch_y_test)

            total_test_loss += test_loss.item()

        average_test_loss = total_test_loss / len(test_loader)
        test_hist.append(average_test_loss)

    print(
        f'Epoch [{epoch + 1}/{num_epochs}] - Training Loss: {average_loss:.4f}, Test Loss: {average_test_loss:.4f}')


KeyboardInterrupt: 